In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
import matplotlib.pyplot as plt
import numpy as np

In [4]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5, stride=(1,1), padding=2)
        self.batch1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d((2,2), stride=(2,2))
        
        self.conv2 = nn.Conv2d(64, 128, 5, stride=(1,1), padding=2)
        self.batch2 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d((2,2), stride=(2,2))

        self.conv3 = nn.Conv2d(128, 256, 3, stride=(1,1), padding=1)
        self.batch3 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d((2,2), stride=(2,2))

        self.conv4 = nn.Conv2d(256, 512, 3, stride=(1,1), padding=1)
        self.batch4 = nn.BatchNorm2d(512)
        
        self.linear1 = nn.Linear(512 * 16 * 16, 1024)
        self.batch5 = nn.BatchNorm2d(1024)
        

    def forward(self, x):
        x = self.pool1(self.batch1(F.relu(self.conv1(x))))
        x = self.pool2(self.batch2(F.relu(self.conv2(x))))
        x = self.pool3(self.batch3(F.relu(self.conv3(x))))
        x = self.batch4(F.relu(self.conv4(x)))
        
        #Flatten
        x = x.view(-1, self.num_flat_features(x))
        
        x = self.batch4(F.relu(self.linear1(x)))
        return x


net = Net().cuda()

In [5]:
print(net)

Net (
  (conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batch1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (pool1): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batch2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
  (pool2): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  (pool3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
  (linear1): Linear (131072 -> 1024)
  (batch5): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True)
)


In [7]:
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-6
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [8]:
%matplotlib inline
def show(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    plt.show()